In [ ]:
import numpy as np
import urllib.request
import os

datafile = 'organamnist'

# Download the dataset to the local folder
if not os.path.isfile(f'./{datafile}.npz'):
    urllib.request.urlretrieve(f'https://zenodo.org/records/10519652/files/{datafile}.npz?download=1', f'{datafile}.npz')

# Load the compressed numpy array file
dataset = np.load(f'./{datafile}.npz')

# The loaded dataset contains each array internally
for key in dataset.keys():
    print(f'dict key: {key:12s}, array shape: {dataset[key].shape}, array datatpye: {dataset[key].dtype}')

In [ ]:
import matplotlib.pyplot as plt

class_ids, class_first_occur = np.unique(dataset['train_labels'], return_index=True)

print(f'This dataset contains {len(class_ids)} classes.')


Nrows = 3; Ncols = 4
fig, ax = plt.subplots( Nrows, Ncols, sharex=True, sharey=True)

for i in range(Nrows):
    for j in range(Ncols):
        if( i*Ncols + j < len(class_ids)):
            idx = class_first_occur[i*Ncols + j]
            label = dataset['train_labels'][idx,0]
            ax[i,j].set_title(f'Class {label}')
            ax[i,j].set_yticks([])
            ax[i,j].set_xticks([])
            ax[i,j].imshow(dataset['train_images'][idx], cmap='gray')
        else:
            ax[i,j].axis('off')
plt.show()


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.datasets import fetch_openml

for key in dataset.keys():
    print(f'dict key: {key:12s}, array shape: {dataset[key].shape}, array dtype: {dataset[key].dtype}')

X = dataset['train_images'].reshape(dataset['train_images'].shape[0], -1)  # Flatten the images
y = dataset['train_labels'].flatten()

kmeans = KMeans(n_clusters=11, random_state=42)
kmeans.fit(X)

cluster_labels = kmeans.predict(X)

pca = PCA(n_components=3)
X_pca = pca.fit_transform(X)

fig = plt.figure(figsize=(15, 7))

ax1 = fig.add_subplot(121, projection='3d')
scatter1 = ax1.scatter(
    X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
    c=y, cmap='tab10', s=10
)
ax1.set_title("3D Plot with True Labels")
ax1.set_xlabel("PCA 1")
ax1.set_ylabel("PCA 2")
ax1.set_zlabel("PCA 3")
plt.colorbar(scatter1, ax=ax1)

ax2 = fig.add_subplot(122, projection='3d')
scatter2 = ax2.scatter(
    X_pca[:, 0], X_pca[:, 1], X_pca[:, 2],
    c=cluster_labels, cmap='tab10', s=10
)
ax2.set_title("3D Plot with Cluster Labels")
ax2.set_xlabel("PCA 1")
ax2.set_ylabel("PCA 2")
ax2.set_zlabel("PCA 3")
plt.colorbar(scatter2, ax=ax2)

plt.show()


In [ ]:
import numpy as np
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.model_selection import GridSearchCV


In [ ]:
#######################################################################################################################################################################################
#for the regression/knn methods

X_train, X_test, y_train, y_test = train_test_split(dataset['train_images'], dataset['train_labels'], random_state=0)  ## if not specified auto splits into 75%  train, 25% test

X_train_flat = X_train.reshape(X_train.shape[0], -1)
X_test_flat = X_test.reshape(X_test.shape[0], -1)

y_test_flat = y_test.reshape(y_test.shape[0], -1)
y_train_flat = y_train.reshape(y_train.shape[0], -1)


pca = PCA(n_components=50)
X_train_pca = pca.fit_transform(X_train_flat)
X_test_pca = pca.transform(X_test_flat)

############################################################################################################################################################################################################
##### for the nn models

X = dataset['train_images']  # Shape (N, H, W) where N is the number of samples
y = dataset['train_labels']  # Shape (N, 1), single-label classes

# Normalize images and convert to torch tensors
X = X.astype(np.float32) / 255.0  # Normalize to [0, 1]
X = torch.tensor(X)  # Convert to tensor

# Flatten the labels to make them 1D
y = y
y = y.flatten()
y = torch.tensor(y, dtype=torch.long)
  # Convert to tensor

# Split the dataset into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# Create DataLoader objects
train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

In [ ]:
knn = KNeighborsClassifier(n_neighbors=1)
knn.fit(X_train_pca, y_train_flat)
y_pred = knn.predict(X_test_pca)

KNN_Training_score = knn.score(X_train_pca, y_train_flat) * 100
KNN_Accuracy_score = knn.score(X_test_pca, y_test_flat) * 100
print(f"train score", KNN_Training_score)
print(f"test score", KNN_Accuracy_score)

In [ ]:
log_reg_model = LogisticRegression(max_iter=1000, multi_class='multinomial')
log_reg_model.fit(X_train_pca, y_train_flat)

y_pred_log_reg = log_reg_model.predict(X_test_pca)
log_reg_train_score = log_reg_model.score(X_train_pca, y_train_flat) * 100
log_reg_accuracy = accuracy_score(y_test_flat, y_pred_log_reg) * 100
print(f"Logistic Regression Train Accuracy: {log_reg_train_score:.4f}")
print(f"Logistic Regression Test Accuracy: {log_reg_accuracy:.4f}")


In [ ]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 11)

    def forward(self, x):
        x = x.unsqueeze(1)
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)

        return x


In [ ]:
cnn_model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_model.parameters(), lr=0.001)

num_epochs = 5
for epoch in range(num_epochs):
    cnn_model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = cnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_loader)
    cnnepoch_acc = correct / total
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {cnnepoch_acc:.2f}%')
cnn_model.eval()
val_correct = 0
val_total = 0
with torch.no_grad():
    for images, labels in val_loader:
        outputs = cnn_model(images)
        _, predicted = torch.max(outputs, 1)

        val_total += labels.size(0)
        val_correct += (predicted == labels).sum().item()

CNNtrain_score = cnnepoch_acc * 100
print(f'Training Accuracy: {CNNtrain_score:.2f}%')
val_accuracy = val_correct / val_total * 100
print(f'Validation Accuracy: {val_accuracy:.2f}%')


In [ ]:
class FCNN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(FCNN, self).__init__()
        self.model = nn.Sequential(
            nn.Linear(input_size, 512),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = x.view(x.size(0), -1)
        return self.model(x)


In [ ]:
input_size = 28 * 28
num_classes = 11
num_epochs = 10
fcnn_model = FCNN(input_size, num_classes)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(fcnn_model.parameters(), lr=0.001)

for epoch in range(num_epochs):
    fcnn_model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for images, labels in train_loader:
        optimizer.zero_grad()
        outputs = fcnn_model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    epoch_loss = running_loss / len(train_loader)
    epoch_acc = correct / total
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.2f}%")

fcnn_model.eval()
fcnnval_correct = 0
fcnnval_total = 0

with torch.no_grad():
    for images, labels in val_loader:
        outputs = fcnn_model(images)
        _, predicted = torch.max(outputs, 1)
        fcnnval_total += labels.size(0)
        fcnnval_correct += (predicted == labels).sum().item()
FCNNTrain_score = epoch_acc * 100
print(f"Training Accuracy: {FCNNTrain_score:.2f}%")
fcnn_accuracy = fcnnval_correct / fcnnval_total * 100
print(f"Validation Accuracy: {fcnn_accuracy:.2f}%")

In [ ]:
# Display accuracies for all models
print(f"Logistic Regression Test Accuracy: {log_reg_accuracy:.4f}%, Logisitic regression training score: {log_reg_train_score:.4f}%")
print(f"FCNN Test Accuracy: {fcnn_accuracy:.4f}% FCNN train accuracy: {FCNNTrain_score:.4f}%")
print(f"CNN Test Accuracy: {val_accuracy:.4f}%, CNN train accuracy: {CNNtrain_score:.4f}%")
print(f"K nearest neighbors Test Accuracy: {KNN_Accuracy_score:.4f}%, KNN training score: {KNN_Training_score:.4f}%")


Logistic Regression Test Accuracy: 74.5168%, Logisitic regression training score: 75.5826%
FCNN Test Accuracy: 92.0440% FCNN train accuracy: 91.3122%
CNN Test Accuracy: 98.4956%, CNN train accuracy: 98.4411%
K nearest neighbors Test Accuracy: 99.8264%, KNN training score: 100.0000%

In [ ]:
import numpy as np
import torch
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix


models = [fcnn_model, cnn_model, knn, log_reg_model]
model_names = ["FCNN", "CNN", "KNN", "Logistic Regression"]
labels = [str(i) for i in range(1, 12)]

fig, axes = plt.subplots(2, 2, figsize=(14, 12))
axes = axes.ravel()

for i, model in enumerate(models):
    if model == knn:
        y_pred = model.predict(X_test_pca)
        cm = confusion_matrix(y_test_flat, y_pred)
    elif model == log_reg_model:
        y_pred = model.predict(X_test_pca)
        cm = confusion_matrix(y_test_flat, y_pred)
    else:
        model.eval()
        preds = []
        true_labels = []

        with torch.no_grad():
            for images, labels in val_loader:
                outputs = model(images)
                _, predicted = torch.max(outputs, 1)

                preds.extend(predicted.cpu().numpy())
                true_labels.extend(labels.cpu().numpy())

        preds = np.array(preds)
        true_labels = np.array(true_labels)

        cm = confusion_matrix(true_labels, preds)

    sns.heatmap(cm, annot=True, fmt="d", cmap="Reds", xticklabels=labels, yticklabels=labels, ax=axes[i])
    axes[i].set_title(f"Confusion Matrix - {model_names[i]}")
    axes[i].set_xlabel("Predicted")
    axes[i].set_ylabel("True")

plt.tight_layout()
plt.show()
